In [1]:
from lark import Lark, Transformer
import joblib 
import rulesets
import importlib.util
importlib.reload(rulesets)

from rulesets import *
from nlp_explanation import *

In [6]:
with open('../config/grammar.txt') as grammar_file:
    grammar = grammar_file.read()
parser = Lark(grammar)
import itertools

In [75]:
rule = joblib.load('../objects/amazon/rules_3_experiment/rule_3_5')
decorador = RuleStringDecorator(rule)

In [40]:
texto = "if story==5 and ((character>=5 and character<=6) then confidence = 0.9"

In [41]:
 #or read==10 then confidence = 0.9"#decorador.getString()

In [76]:
texto = decorador.getString()

In [77]:
texto

'if character==6 then confidence = 0.006549339577768834'

In [7]:
tree = parser.parse(texto)

In [23]:
#@lark.v_args(inline=True)
class myStack:
    def __init__(self):
        self.container = []  # You don't want to assign [] to self - when you do that, you're just assigning to a new local variable called `self`.  You want your stack to *have* a list, not *be* a list.
        
    def isEmpty(self):
        return self.size() == 0   # While there's nothing wrong with self.container == [], there is a builtin function for that purpose, so we may as well use it.  And while we're at it, it's often nice to use your own internal functions, so behavior is more consistent.

    def push(self, item):
        self.container.append(item)  # appending to the *container*, not the instance itself.

    def pop(self):
        if self.isEmpty():
            return ''
        return self.container.pop()  # pop from the container, this was fixed from the old version which was wrong
    
    def peek(self):
        if self.isEmpty():
            raise Exception("Stack empty!")
        return self.container[-1]  # View element at top of the stack

    def size(self):
        return len(self.container)  # length of the container

    def show(self):
        return self.container  # display the entire stack as list
from queue import SimpleQueue

class MyTransformer(Transformer):

    def __init__(self,positive_class=''):
        self.nplist = []
        self.items_visitados = []
        self.last_visited=myStack()
        self.cont = 0
        self.explicacion = ''
        self.relevancia1=''
        self.relevancia2=''
        self.positive_class=positive_class
        self.message = ''
    def start(self, items):
        caracter = 'The rule states that if the importance/relevance of the terms are as follows: '
        int = 1
        while self.last_visited.isEmpty() == False:
        
            caracter+=self.last_visited.pop()+' '
        
        if "is at levels ranging from moderately relevant to moderately relevant" in caracter:
            caracter=caracter.replace('is at levels ranging from moderately relevant to moderately relevant',' is moderately relevant')
        elif "is at levels ranging from not relevant to not relevant" in caracter:
            caracter=caracter.replace("is at levels ranging from not relevant to not relevant", "is not relevant")
        elif "is at levels ranging from not very relevant to not very relevant" in caracter:
            caracter=caracter.replace("is at levels ranging from not very relevant to not very relevant", "is not very relevant")
        elif "is at levels ranging from relevant to relevant" in caracter:
            caracter=caracter.replace("is at levels ranging from relevant to relevant","is relevant")
        elif "is at levels ranging from very relevant to very relevant" in caracter:
            caracter=caracter.replace("is at levels ranging from very relevant to very relevant","is very relevant")
        elif "is at levels ranging from very at maximum level of relevance to at maximum level of relevance" in caracter:
            caracter=caracter.replace("is at levels ranging from very at maximum level of relevance to at maximum level of relevance","is at maximum level of relevance")
        else:
            caracter+=", then the text identified by this rule is likely to belong to the possitive class "+self.positive_class
        print(caracter)
        
        self.message = caracter
        
#     def expresion_compuesta(self,items):
#         for item in items:
#             if 

    def devuelveMensaje(self):
        return self.message
    
    def expresion_and(self,items):
        self.cont = self.cont + 1
        expresion2 = self.last_visited.pop()
        expresion1 = self.last_visited.pop()
        #print (expresion1 + ' _ Y _ '+ expresion2)
        print('\nEXPRESION AND '+str(self.cont)+': '+expresion2 + ' _ Y _ '+ expresion1)

        if len(expresion1)>0:
            
            if expresion2.startswith('to') or expresion2.startswith('(to'):
                if expresion2.startswith('('):
                    expresion2 = expresion2[1:]
                    expresion2 = expresion2[:-1]
                self.last_visited.push(''+expresion1 + ' '+ expresion2+'')
            else:    
                self.last_visited.push(''+expresion1 + ' and '+ expresion2+'')
        else:
            self.last_visited.push(expresion2)        
        
        self.relevancia1=''
        self.relevancia2=''
        #self.last_visited.put( 'AND',block=False )
      
    def expresion_or (self,items):
        self.cont = self.cont + 1

        expresion2 = self.last_visited.pop()
        expresion1 = self.last_visited.pop()

        if  len(expresion1)>0:
            print('\n EXPRESION OR '+str(self.cont)+': '+expresion1 + ' or '+ expresion2)
            print('range true '+str(self.relevancia1)+'----'+str(self.relevancia2))

            if self.relevancia1 == self.relevancia2 and len(self.relevancia1)>0:
                print('Haciendo push de '+expresion1)
                self.last_visited.push(expresion1)
            else:
                print('Haciendo push de '+'('+expresion1 + ' or '+ expresion2+')')

                self.last_visited.push(''+expresion1 + ' or '+ expresion2+'')
        else:
            self.last_visited.push(expresion2)

        self.relevancia1=''
        self.relevancia2=''
#print (expresion1 + ' _ O _ '+ expresion2)
    def expresion (self, items):
        self.cont = self.cont + 1

        self.rangeFrom=False
        self.rangeTo=False
        if items[0]!='(':
            termino = items[0]
            operator = items[1]
            rele = int(str(items[2]))
            operador = ''
            lpar = ''
            rpar=''#items[len(items)-1]
            #if items[len(items)-1]=='':
                #rpar=''
        else:
            termino = items[1]
            operator = items[2]
            rele = int(str(items[3]))
            operador = ''
            lpar = '('
            rpar=''#items[len(items)-1]
            #if items[len(items)-1]=='': 
            #    rpar=''   
        print ('\n'+str(self.cont)+' analizado: ' + termino+operator+str(rele)+'\n') 
        if (operator=='>'):
            operador = 'is at levels ranging from'
            self.rangeFrom = True
            self.rangeTo=False
            print('range from!!!!')
        elif operator=='<' or operator =='<=':
            operador='to'
            self.rangeTo=True
            self.rangeFrom = False
            print('range To!!!!!!!')
        elif operator=='==':
            operador = ' is  '
        elif operator=='>=':
            operador = 'is at levels ranging from'
            self.rangeFrom = True
            self.rangeTo = False
            print('range from!!!!')

        else:
            operador = ''
        
        if operador=='==':
            self.last_visited=''
        if  rele>=0 and rele<=2:
            relevancia = 'not relevant'
            if self.rangeFrom==True:
                self.relevancia1=copy.deepcopy(relevancia)
                print('si estoy metiendo self relevancia1'+self.relevancia1)

            if self.rangeTo==True:
                self.relevancia2 = copy.deepcopy(relevancia)
                
        elif rele>=3 and rele<5:
            relevancia = 'not very relevant'
            if self.rangeFrom==True:
                self.relevancia1=copy.deepcopy(relevancia)
                print('si estoy metiendo self relevancia1'+self.relevancia1)

            if self.rangeTo==True:
                self.relevancia2 = copy.deepcopy(relevancia)
        elif rele>=5 and rele<7:
            relevancia = 'moderately relevant'

            if self.rangeFrom==True:
                self.relevancia1=copy.deepcopy(relevancia)
                print('si estoy metiendo self relevancia1'+self.relevancia1)

            if self.rangeTo==True:
                self.relevancia2 = copy.deepcopy(relevancia)
        elif rele>=6 and rele<8:
            relevancia = 'relevant'
            if self.rangeFrom==True:
                self.relevancia1=copy.deepcopy(relevancia)
                print('si estoy metiendo self relevancia1'+self.relevancia1)

            if self.rangeTo==True:
                self.relevancia2 = copy.deepcopy(relevancia)
        
        elif rele>=8 and rele<=9:
            relevancia = 'very relevant'
            if self.rangeFrom==True:
                self.relevancia1=copy.deepcopy(relevancia)
                print('si estoy metiendo self relevancia1'+self.relevancia1)

            if self.rangeTo==True:
                self.relevancia2 = copy.deepcopy(relevancia)
        else:
            relevancia = 'at maximum level of relevance'
            if self.rangeFrom==True:
                self.relevancia1=copy.deepcopy(relevancia)
                print('si estoy metiendo self relevancia1'+self.relevancia1)
            if self.rangeTo==True:
                self.relevancia2 = copy.deepcopy(relevancia)
            #if termino in self.items_visitados:
        if operator=='<' or operator =='<=':
            if self.rangeTo==True:
                if self.relevancia1.strip()==self.relevancia2.strip():
                    operador = ' is '
                    anterior = self.last_visited.pop()
                    if anterior.startswith('('):
                        lpar = '('
                    
                    print('OJITO QUE ME PULO '+anterior)
                    self.relevancia2 =''
                    print ('Adding simplified to'+ termino + ' '+ operador +' '+ relevancia+')' )
                    self.last_visited.push (''+ termino + ' '+ operador +' '+ relevancia+''  )
                else:
                    print('Adding not simplified: r1'+self.relevancia1+ '- r2:'+self.relevancia2+'::'+lpar+operador +' '+ relevancia+rpar)
                    self.last_visited.push (''+operador +' '+ relevancia+'')
                
        else:
            #if rangeFrom==True:
            #    rpar=''
            #else:
            #    rpar=''
            print('Adding '+lpar+ termino + ' '+ operador +' '+ relevancia+rpar)
            self.last_visited.push (''+ termino + ' '+ operador +' '+ relevancia+''  ) #range from no meter relevancia

    def transform(self, tree):
        self._transform_tree(tree)
        return self.nplist

In [45]:
transformer = MyTransformer('Book Review')
transformer.transform(tree)
print (transformer.devuelveMensaje())


1 analizado: book==4

Adding book  is   not very relevant

2 analizado: read>=7

range from!!!!
si estoy metiendo self relevancia1relevant
Adding read is at levels ranging from relevant

3 analizado: read<=8

range To!!!!!!!
Adding not simplified: r1relevant- r2:very relevant::to very relevant

4 analizado: read==10

Adding read  is   at maximum level of relevance

 EXPRESION OR 5: to very relevant or read  is   at maximum level of relevance
range true relevant----very relevant
Haciendo push de (to very relevant or read  is   at maximum level of relevance)

EXPRESION AND 6: to very relevant or read  is   at maximum level of relevance _ Y _ read is at levels ranging from relevant

EXPRESION AND 7: read is at levels ranging from relevant to very relevant or read  is   at maximum level of relevance _ Y _ book  is   not very relevant
The rule states that if the importance/relevance of the terms are as follows: book  is   not very relevant and read is at levels ranging from relevant to ver

In [25]:
from spotlight import *
class TextAnalyzer(object):
    def __init__(self,nlp):
        self.nlp = nlp 
        
    # allow the class instance to be called just like
    # just like a function and applies the preprocessing and
    # tokenize the document
    @staticmethod      
    def remove_special_lines(texto):
        texto = re.sub("^upright=.*[\r|\n]", '', texto)
        texto = re.sub("^upright = .*[\r|\n]", '', texto)
        texto = re.sub("Category:.*[\r|\n]",'',texto)
        texto = re.sub("Cat\D*:.*[\r|\n]",'',texto)
        texto = re.sub("[[][\d]+[]]",'',texto)
        texto = re.sub("thumb",'',texto)
        texto = re.sub("[|]",'',texto)
        texto = re.sub("\d+px",'',texto)
        return (texto)
    @staticmethod
    def strip_formatting(string):
        string = string.lower()
        string = re.sub(r"([.!?,;-_'/|()]=-<>+*`)", r"", string)
        string = re.sub(r'https?:\/\/.*?[\s]', '', string) 
        return string

    def get_nlp(self):
        return self.nlp
    
    def __call__(self, doc):
        tokens = nlp(doc)
        lemmatized_tokens = [(token.lemma_.lower()) for token in tokens
                                                   if not (token.is_stop or token.is_punct)]
            
        return(lemmatized_tokens)
    
    def is_present (self,word,text):
        lemmatized_tokens =  lambda text: " ".join(token.lemma_.lower() for token in nlp(text) if not (token.is_stop or token.is_punct))
        normalizado = lemmatized_tokens(text)    
        return (word in (normalizado))

class SemanticAnalyzer(TextAnalyzer):
    def __init__(self,nlp,endpoint='http://192.168.99.100:2222/rest/annotate/',soporte=1,confianza=0.1,umbral=0.1):
        super().__init__(nlp)
        self.endpoint = endpoint
        self.soporte=soporte
        self.confianza = confianza
        self.alfa = umbral
    
    def __call__(self, doc):
        try:
            annotations = spotlight.annotate(self.endpoint,
                                     doc,
                                      confidence=self.confianza, support=self.soporte, spotter='Default')
            diccionario =  dict()
            for annotation in annotations:
                lista = list(annotation.items())
                print(lista)
                URI = lista[0]
                key = lista[3]
                score = lista[5]
               # if (score[1]>self.alfa):
                diccionario[key[1]]=URI[1]
        
            return(diccionario)
        except Exception as ex:
            print(ex)

In [26]:
from nltk.corpus import wordnet as wn


In [27]:
#cargamos dbpedia y sumo

class OntologyOracle(object):
    def __init__(self,nlp,dict_onto,dict_graph):

        self.dict_onto = dict_onto
        self.dict_graph = dict_graph
        
        self.prefijos = """  PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
                                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                                PREFIX dbr:    <http://dbpedia.org/resource/>
                                PREFIX dbo:    <http://dbpedia.org/ontology/>
                                PREFIX dct:    <http://purl.org/dc/terms/>
                                PREFIX owl:    <http://www.w3.org/2002/07/owl#>
                                PREFIX rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                                PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
                                PREFIX schema: <http://schema.org/>
                                PREFIX skos:   <http://www.w3.org/2004/02/skos/core#>
                                PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#>
                                PREFIX SUMO: <http://www.adampease.org/OP/SUMO.owl#>
                                PREFIX SUMOR: <http://www.ontologyportal.org/SUMO.owl#>
                            """
        self.sa = SemanticAnalyzer(nlp)
      
    
    
    def isSubclassOf(self,term1,term2,isDbo):
        
        consulta = self.prefijos + """
        ASK  { 
                """+term1+""" rdfs:subClassOf """+term2+""" }"""
          
        if (isDbo==False):
            return (list(self.dict_graph.get('SUMO').query(consulta)))  
        else:
            return (list(self.dict_graph.get('dbo').query(consulta)))
    
    
    def isSuperclassOf (self,term1,term2,isDbo):
        consulta = self.prefijos + """
        ASK  {  
                """+term2+""" rdfs:subClassOf """+term1+""" }"""
          
        if (isDbo==False):
            return (list(self.dict_graph.get('SUMO').query(consulta)))  
        else:
            return (list(self.dict_graph.get('dbo').query(consulta)))
    
        
    def areRelated (self,term1,term2,isDbo):
        consulta = self.prefijos + """
        ASK  {  """+term1+""" ?property """+term2+""" }"""
          
        if (isDbo==False):
            return (list(self.dict_graph.get('SUMO').query(consulta)))  
        else:
            return (list(self.dict_graph.get('dbo').query(consulta)))
        
    def getRelation(self,term1,term2,isDbo):
        consulta = self.prefijos + """
        select ?property where  {  """+term1+""" ?property """+term2+""" }"""
          
        if (isDbo==False):
            return (list(self.dict_graph.get('SUMO').query(consulta)))  
        else:
            return (list(self.dict_graph.get('dbo').query(consulta)))
    
    def isDbo (self,term):
        res = self.dict_onto.get('dbo').search(label=term,_case_sensitive=False)
        if (len(res)>0):
            return True
        res = self.dict_onto.get('SUMO').search(label=term,_case_sensitive=False)
        if (len(res)>0):
            return False
        else:
            return None

    def _getBaseConcept(self,term,isDbo):
        if (isDbo):
            res = self.dict_onto.get('dbo').search(label=term,_case_sensitive=False)
            if (len(res)>0):
                return str(res[0]).replace('.',':')
        
        else:
            res = self.dict_onto.get('SUMO').search(label=term,_case_sensitive=False)
        
        if (len(res)>0):
            return str(res[0]).replace('.',':')
        else:
            return None
        
    def _getHierarchy(self,concept,isDbo,isSuperclass):
        
        consulta = self.prefijos + """SELECT ?x
            WHERE {
                ?x a owl:Class .
                ?x rdfs:subClassOf """+concept+"""
                }"""
        columna = 'superclass'
        if isSuperclass == False:
            columna='subclass'
            consulta = self.prefijos + """SELECT ?x
            WHERE {
                ?x a owl:Class .
                """+concept+""" rdfs:subClassOf ?x 
                }"""
                
        if (isDbo==False):
            resultado =  (list(self.dict_graph.get('SUMO').query(consulta)))  
        else:
            resultado =  (list(self.dict_graph.get('dbo').query(consulta)))
           
        newresult = [r for res in resultado for r in res ]
        print(newresult)
        columnas = [columna]
        res = pd.DataFrame(data=newresult,columns=columnas)
        return newresult
    
    def _getRelationships(self,concept,isDbo):
        consulta = self.prefijos + """select distinct  ?property ?value where {
                  """+concept+""" ?property ?value .
                  filter ( ?property not in ( rdf:type ) )
                   filter ( ?property not in ( rdfs:label ) )
                   filter ( ?property not in ( rdf:type ) )
                   filter ( ?property not in ( rdfs:isDefinedBy ) )
                   filter ( ?property not in ( SUMOR:externalImage ) )
                   filter ( ?property not in ( SUMOR:axiom ) )
                   
                  optional {?property rdfs:comment ?comment}
                  optional {?property rdfs:range ?range} 
                  optional {?property rdfs:domain ?domain} 
                }
        """
        #print(consulta)
        if (isDbo==False):
            resultado =  (list(self.dict_graph.get('SUMO').query(consulta)))  
            res = pd.DataFrame(data = resultado, columns = ['property','element'])
            wordnetlist = list()
            for wne in res.element:
                uri = wne.n3()
                uri = uri.replace('http://www.adampease.org/OP/wn','./ontologias/WordNet.owl')
                wnconsulta = self.prefijos + """select distinct ?label ?comment where {
               """+uri+"""  rdf:type owl:Thing .
               """+uri+"""   rdfs:label ?label .
                """+uri+"""   rdfs:comment ?comment .
                
                
                }"""
                
                resultado = self.dict_graph.get('wn').query(wnconsulta)

                wordnetlist.append([r for res in resultado for r in res]) 
            res['explicacion']=wordnetlist

        else:
            resultado = (list(self.dict_graph.get('dbo').query(consulta)))
            res = pd.DataFrame(data = resultado, columns = ['property','element'])

        res['term'] = concept
        res.set_index(res.term)
        return res #(res.to_json())

            
    def getSemanticsOfTerm(self,term,isDbo=None):
        
        if isDbo == None:
            isDbo = self.isDbo(term)
        #if (isDbo):
        #    resources = {} #self.sa(term)
        #else:
        #    resources = {}
        if isDbo is None:
            #devolvemos semantica de WN
            wordnetSemantics = list()
            sentidos = wn.synsets(term)
            wordnetSemantics.append([(lemma.name(), sysnet.definition()) for sysnet in sentidos for lemma in sysnet.lemmas()])
            resultado = pd.DataFrame(data=wordnetSemantics, columns=['termino','significado'])
            return False, resultado
        
        concept = self._getBaseConcept(term,isDbo)
        if concept is None:
            return None
        superclasses = self._getHierarchy(str(concept),isDbo,False)
        subclasses = self._getHierarchy(str(concept), isDbo,True)
        relationships = self._getRelationships(str(concept),isDbo)
        
        termino = dict({'concepto':concept,'padres':superclasses,'hijos':subclasses,'relaciones':relationships})
        
        return True,termino
    
          

In [74]:
class SemanticRuleExplainer (object):
    
    def __init__(self,n_terms=3):
        self.n_terms = n_terms
    
    def construirRelatoTermino(self,alfred,term,isDbo): 
        relatos=''
        sentidos = wn.synsets(term)
        if sentidos is None:
            relatos +='WordNet does not contains any sense for :'+term+'. Check if it is a domain specific term or is a redaction issue\n'
        else:
            relatos += 'Term: '+term+' has been found in WordNet. Retrieving '+str(self.n_terms)+' definitions\n'
            cont = 0
            for synset in sentidos:
                for lemma in synset.lemmas():
                    relatos+='\t'+str(cont)+'. Lemma sense for '+term+': '+lemma.name()+', Definition: '+ synset.definition()+'\n'
                    cont = cont + 1 
                    if cont >= self.n_terms:
                        break
                if cont >= self.n_terms:
                    break
                       
       
        try:
            valor,dboTerms = alfred.getSemanticsOfTerm(term,False)

            #print(dboTerms)
            #print(type(dboTerms))
            if dboTerms is not None:
                relatos+='\nThe concept has been found in SUMO ontology: '+dboTerms.get('concepto')+'\n'

                relatos +='Find below the superclasses of the concept:'

                elementos = dboTerms.get('padres')
                for elemento in elementos:
                    print ('\t'+elemento)

        #        print('Hijos: ')
        #        mdFile.new_header(level=4, title='Hijos de: '+term)

        #        elementos = dboTerms.get('hijos')
        #        listah = list()
        #        listah = [elemento for elemento in elementos]
        #        mdFile.new_list(items=listah)



        #        relatos+= 'Some relations have been found: '

        #        relaciones = dboTerms.get('relaciones')
        #        lista  = ["Property", "Descriptions"]
        #        fila = 1

        #        for index, rows in relaciones.iterrows():
        #            relatos+='\t{0}: {1} {2}\n'.format(str(fila),rows['property'],rows['explicacion'].__str__())
        #            fila = fila+1
        except:
            relatos +='The concept related to term '+term+' was not found in SUMO'

      
        if isDbo == True:
            #relatos+=('\n********************Retrieving semantics in DBO********************\n')
            try:
                valor,dboTerms = alfred.getSemanticsOfTerm(term,True)

                #print(dboTerms)
                #print(type(dboTerms))
                if dboTerms is not None:
                    relatos+='\nThe concept has been found in DBpedia: '+dboTerms.get('concepto')+'\n'

                    relatos +='Find below the superclasses of the concept:'

                    elementos = dboTerms.get('padres')
                    for elemento in elementos:
                        print ('\t'+elemento)
            except:
                relatos +='The concept related to term '+term+' was not found in DBpedia'

#                relatos+= 'Some relations have been found: '

#                relaciones = dboTerms.get('relaciones')
#                lista  = ["Property", "Descriptions"]
#                fila = 1

#                for index, rows in relaciones.iterrows():
#                    relatos+='\t{0}: {1} {2}\n'.format(str(fila),rows['property'],rows['explicacion'].__str__())
#                    fila = fila+1

            
        return relatos

In [38]:
import spacy
import time 
from spacy_langdetect import LanguageDetector
nlp = spacy.load("en_core_web_md")

In [39]:
myworld1 = World()
ontsumo = sumo =myworld1.get_ontology("../data/ontologias/SUMO.owl")

In [40]:
ontsumo.base_iri="http://www.adampease.org/OP/SUMO.owl#"
ontsumo.name='SUMO'
sumo =ontsumo.load() #http://www.adampease.org/OP/SUMO.owl
graphsumo = myworld1.as_rdflib_graph()

In [41]:
myworld2 = World()
ontdbpedia = dbpedia=myworld2.get_ontology("../data/ontologies/dbpedia_2016-10.owl")

In [33]:
dbpedia = ontdbpedia.load()

ontdbpedia.base_iri = "http://dbpedia.org/ontology/"
ontdbpedia.name='dbo'
graphdbo = myworld2.as_rdflib_graph()

* Owlready2 * WARNING: ObjectProperty http://dbpedia.org/ontology/senator belongs to more than one entity types: [owl.ObjectProperty, dbpedia_2016-10.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types: [owl.ObjectProperty, dbpedia_2016-10.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://dbpedia.org/ontology/productShape belongs to more than one entity types: [owl.DatatypeProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://dbpedia.org/ontology/latinName belongs to more than one entity types: [owl.DatatypeProperty, dbpedia_2016-10.Name]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types: [owl.DatatypeProperty, dbpedia_2016-10.LanguageCode]; I'm trying to fix it...
* Owlready

In [34]:
myworld3 = World()
ontwordnet = myworld3.get_ontology("../data/ontologies/WordNet.owl")


wordnet = ontwordnet.load()
ontwordnet.base_iri = "http://www.adampease.org/OP/WordNet.owl#"
ontwordnet.name='wn'
graphwordnet = myworld3.as_rdflib_graph()


    

In [35]:
    

dict_onto = dict([('dbo',dbpedia),('SUMO',sumo),('wn',wordnet)])
dict_graph = dict([('dbo',graphdbo),('SUMO',graphsumo),('wn',graphwordnet)])
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/rdelaguila/anaconda3/lib/python3.7/site-packages/owlready2/hermit:/home/rdelaguila/anaconda3/lib/python3.7/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpp0x5ewr6
* Owlready2 * HermiT took 2.330838203430176 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [67]:
alfred = OntologyOracle(nlp,dict_onto,dict_graph)

In [68]:
try:
    valor, dboTerms = alfred.getSemanticsOfTerm('read',False)
except TypeError:
    print('Nones')

Nones


In [72]:
platon = SemanticRuleExplainer()
relato = platon.construirRelatoTermino(alfred,'read',False)

In [73]:
print (relato)

Term: read has been found in WordNet. Retrieving 3 definitions
	0. Lemma sense for read: read, Definition: something that is read
	1. Lemma sense for read: read, Definition: interpret something that is written or printed
	2. Lemma sense for read: read, Definition: have or contain a certain wording or form
	3. Lemma sense for read: read, Definition: look at, interpret, and say out loud something that is written or printed
	4. Lemma sense for read: read, Definition: obtain data from magnetic tapes
	5. Lemma sense for read: read, Definition: interpret the significance of, as of palms, tea leaves, intestines, the sky; also of human behavior
	6. Lemma sense for read: take, Definition: interpret something in a certain way; convey a particular meaning or impression
	7. Lemma sense for read: learn, Definition: be a student of a certain subject
	8. Lemma sense for read: read, Definition: indicate a certain reading; of gauges and instruments
	9. Lemma sense for read: read, Definition: audition f